# Tessellation point-wise evaluation

    - Read files in each directory and create patient specific dictionaries.
    - Make a list of the points:
        - tessellation based points: territory/other.
        - ischemic/non-ischemic.
    - find shared points.
    - Confusion matrix. 

In [19]:
import os
import glob
import vtk
import numpy as np
from utilities import ReadVTPFile

In [ ]:
main_dir = "/Volumes/T7_Research/CABG_Backup/04_Tesselation/SurfaceProjections"
LAD_dir = os.path.join(main_dir,"LAD")
LCx_dir = os.path.join(main_dir,"LCx")
RCA_dir = os.path.join(main_dir,"RCA")

LAD_files = sorted(glob.glob(f"{LAD_dir}/*.vtp"))
LCx_files = sorted(glob.glob(f"{LCx_dir}/*.vtp"))
RCA_files = sorted(glob.glob(f"{RCA_dir}/*.vtp"))

In [16]:
tag_list = ["Myocardium", "Tessellation", "Ischemic"]


LAD_dict = dict()
LAD_patient_list = list()

for file in LAD_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    patient_name, _ = file_name.split('_')
    LAD_patient_list.append(patient_name)

LAD_patient_list = set(LAD_patient_list)
print(LAD_patient_list)

for patient_name in LAD_patient_list:
    inner_dict = dict()
    for tag in tag_list:
        file_name = f"{patient_name}_{tag}.vtp"
        inner_dict[tag] = os.path.join(LAD_dir, file_name)
    LAD_dict[patient_name] = inner_dict



{'VA13', 'SU13', 'SU03', 'SU20', 'SU26', 'VA10', 'VA20'}


In [17]:
LCx_dict = dict()
LCx_patient_list = list()

for file in LCx_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    patient_name, _ = file_name.split('_')
    LCx_patient_list.append(patient_name)

LCx_patient_list = set(LCx_patient_list)
print(LCx_patient_list)

for patient_name in LCx_patient_list:
    inner_dict = dict()
    for tag in tag_list:
        file_name = f"{patient_name}_{tag}.vtp"
        inner_dict[tag] = os.path.join(LCx_dir, file_name)
    LCx_dict[patient_name] = inner_dict


{'VA08', 'SU04', 'SU24', 'SU30', 'VA04', 'FS03', 'SU11', 'SU06', 'SU26', 'VA07'}


In [18]:
RCA_dict = dict()
RCA_patient_list = list()

for file in RCA_files:
    file_name = os.path.splitext(os.path.basename(file))[0]
    patient_name, _ = file_name.split('_')
    RCA_patient_list.append(patient_name)

RCA_patient_list = set(RCA_patient_list)
print(RCA_patient_list)

for patient_name in RCA_patient_list:
    inner_dict = dict()
    for tag in tag_list:
        file_name = f"{patient_name}_{tag}.vtp"
        inner_dict[tag] = os.path.join(RCA_dir, file_name)
    RCA_dict[patient_name] = inner_dict


{'VA15', 'VA17', 'VA16', 'SU13', 'SU07', 'VA14', 'SU03', 'SU26', 'SU14', 'VA20'}


In [47]:
def GetPoints(polydata):
    points = []
    for i in range(polydata.GetNumberOfPoints()):
        p = polydata.GetPoint(i)
        # Round to 3 decimal places (e.g., 1mm precision)
        points.append((round(p[0], 3), round(p[1], 3), round(p[2], 3)))
    return points

TP1 = 0; TN1 = 0; FN1 = 0; FP1 = 0
for patient_name in LAD_dict:
    Myocardium = ReadVTPFile(LAD_dict[patient_name]["Myocardium"])
    Tessellation = ReadVTPFile(LAD_dict[patient_name]["Tessellation"])
    Ischemic = ReadVTPFile(LAD_dict[patient_name]["Ischemic"])

    Myocardium_points = set(GetPoints(Myocardium))
    Tessellation_points = set(GetPoints(Tessellation))
    Ischemic_points = set(GetPoints(Ischemic))

    Non_Tessellation_points = set([item for item in Myocardium_points if item not in Tessellation_points])
    Non_Ischemic_points = set([item for item in Myocardium_points if item not in Ischemic_points])

    TP1 += len(list(Tessellation_points & Ischemic_points))
    TN1 += len(list(Non_Tessellation_points & Non_Ischemic_points))
    FP1 += len(list(Tessellation_points & Non_Ischemic_points))
    FN1 += len(list(Non_Tessellation_points & Ischemic_points))

print(TP1, FP1)
print(FN1, TN1)

sensitivity = TP1/(TP1+FN1)
specificity = TN1/(TN1+FP1)
percision = TP1/(TP1+FP1)
FalsePositiveRate = FP1/(FP1 + TN1)

print(f"Sensitivity: {sensitivity:.2%}")
print(f"Specificity: {specificity:.2%}")
print(f"Precision:   {percision:.2%}") # Note: TP / (TP + FP)
print(f"FPR:         {FalsePositiveRate:.2%}")



641150 174980
106406 998751
Sensitivity: 85.77%
Specificity: 85.09%
Precision:   78.56%
FPR:         14.91%


In [48]:
TP2 = 0; TN2 = 0; FN2 = 0; FP2 = 0
for patient_name in LCx_dict:
    Myocardium = ReadVTPFile(LCx_dict[patient_name]["Myocardium"])
    Tessellation = ReadVTPFile(LCx_dict[patient_name]["Tessellation"])
    Ischemic = ReadVTPFile(LCx_dict[patient_name]["Ischemic"])

    Myocardium_points = set(GetPoints(Myocardium))
    Tessellation_points = set(GetPoints(Tessellation))
    Ischemic_points = set(GetPoints(Ischemic))

    Non_Tessellation_points = set([item for item in Myocardium_points if item not in Tessellation_points])
    Non_Ischemic_points = set([item for item in Myocardium_points if item not in Ischemic_points])

    TP2 += len(list(Tessellation_points & Ischemic_points))
    TN2 += len(list(Non_Tessellation_points & Non_Ischemic_points))
    FP2 += len(list(Tessellation_points & Non_Ischemic_points))
    FN2 += len(list(Non_Tessellation_points & Ischemic_points))

print(TP2, FP2)
print(FN2, TN2)

sensitivity = TP2/(TP2+FN2)
specificity = TN2/(TN2+FP2)
percision = TP2/(TP2+FP2)
FalsePositiveRate = FP2/(FP2 + TN2)

print(f"Sensitivity: {sensitivity:.2%}")
print(f"Specificity: {specificity:.2%}")
print(f"Precision:   {percision:.2%}") # Note: TP / (TP + FP)
print(f"FPR:         {FalsePositiveRate:.2%}")



441030 226360
194280 1872915
Sensitivity: 69.42%
Specificity: 89.22%
Precision:   66.08%
FPR:         10.78%


In [49]:
TP3 = 0; TN3 = 0; FN3 = 0; FP3 = 0
for patient_name in RCA_dict:
    Myocardium = ReadVTPFile(RCA_dict[patient_name]["Myocardium"])
    Tessellation = ReadVTPFile(RCA_dict[patient_name]["Tessellation"])
    Ischemic = ReadVTPFile(RCA_dict[patient_name]["Ischemic"])

    Myocardium_points = set(GetPoints(Myocardium))
    Tessellation_points = set(GetPoints(Tessellation))
    Ischemic_points = set(GetPoints(Ischemic))

    Non_Tessellation_points = set([item for item in Myocardium_points if item not in Tessellation_points])
    Non_Ischemic_points = set([item for item in Myocardium_points if item not in Ischemic_points])

    TP3 += len(list(Tessellation_points & Ischemic_points))
    TN3 += len(list(Non_Tessellation_points & Non_Ischemic_points))
    FP3 += len(list(Tessellation_points & Non_Ischemic_points))
    FN3 += len(list(Non_Tessellation_points & Ischemic_points))

print(TP3, FP3)
print(FN3, TN3)

sensitivity = TP3/(TP3+FN3)
specificity = TN3/(TN3+FP3)
percision = TP3/(TP3+FP3)
FalsePositiveRate = FP3/(FP3 + TN3)

print(f"Sensitivity: {sensitivity:.2%}")
print(f"Specificity: {specificity:.2%}")
print(f"Precision:   {percision:.2%}") # Note: TP / (TP + FP)
print(f"FPR:         {FalsePositiveRate:.2%}")



465271 167586
157259 1847737
Sensitivity: 74.74%
Specificity: 91.68%
Precision:   73.52%
FPR:         8.32%


In [50]:
TP = TP1 + TP2 + TP3
TN = TN1 + TN2 + TN3
FP = FP1 + FP2 + FP3
FN = FN1 + FN2 + FN3

sensitivity = TP/(TP+FN)
specificity = TN/(TN+FP)
percision = TP/(TP+FP)
FalsePositiveRate = FP/(FP + TN)

print(f"Sensitivity: {sensitivity:.2%}")
print(f"Specificity: {specificity:.2%}")
print(f"Precision:   {percision:.2%}") # Note: TP / (TP + FP)
print(f"FPR:         {FalsePositiveRate:.2%}")

Sensitivity: 77.16%
Specificity: 89.24%
Precision:   73.12%
FPR:         10.76%
